AMIE: https://github.com/dig-team/amie 
    Using `amie3.jar` file to provide KG in the form of TSV file and generate rules (Y)

SPaRKLE: https://github.com/SDM-TIB/SPARKLE/tree/main/AMIE_SPaRKLE
    generating predictions of the rules (Y) using amie_sparkle.py
    for generating the predictions just by modifying the input.json file you can execute amie_sparkle.py 





# use AMIE to mine the rules and store into a database

In [1]:
import os
import subprocess

# Get the current directory where the notebook is running
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Print current directory for debugging
print("Current directory:", current_dir)

# Construct the path to the .jar and .tsv files
jar_path = os.path.join(current_dir, 'amie-dev.jar')
tsv_path = os.path.join(current_dir, 'NSCLP.tsv')
# tsv_path = os.path.join(current_dir, 'french_training.tsv')

# Print paths to check correctness
print("Jar path:", jar_path)
print("TSV path:", tsv_path)

# Command to run the Java program
command = f'java -jar "{jar_path}" "{tsv_path}" -const'

# Use subprocess to run the command
process = subprocess.run(command, shell=True, text=True, capture_output=True)

# Print the output and error if any
print("Output:", process.stdout)
print("Error:", process.stderr)

rules = process.stdout


Current directory: /Users/jason/Documents/Coding Projects/KPI3/KDD HealthCareAI Extension/rule_mining
Jar path: /Users/jason/Documents/Coding Projects/KPI3/KDD HealthCareAI Extension/rule_mining/amie-dev.jar
TSV path: /Users/jason/Documents/Coding Projects/KPI3/KDD HealthCareAI Extension/rule_mining/NSCLP.tsv
Output: Assuming rdf:type as type relation
Loading files... 
  Starting NSCLP.tsv
  Finished NSCLP.tsv, still running: 0
Loaded 18715 facts in 0.20 s using -17 MB
Using HeadCoverage as pruning metric with minimum threshold 0.01
Using recursivity limit 3
Lazy mining assistant that stops counting when the denominator gets too high
No minimum threshold on standard confidence
Filtering on PCA confidence with minimum threshold 0.1
Constants in the arguments of relations are enabled
Lossless (query refinement) heuristics enabled
MRT calls: 0
Starting the mining phase... Using 10 threads
Rule	Head Coverage	Std Confidence	PCA Confidence	Positive Examples	Body size	PCA Body size	Functional

In [2]:
rest = []
start_line = False

for l in rules.split('\n'):
    if l.startswith('Rule'):
        start_line = True
        rest.append(l)
        continue
    if l.startswith("Mining done"):
        start_line = False
    if start_line:
        if l.startswith('?'):
            rest.append(l)
# rest
# print(len(rest))

In [3]:
import mysql.connector
import pandas as pd

def create_mysql_database(text_list, db_name, table_name, user, password, host='localhost'):
    # Extract the header (first line) and the data (remaining lines)
    header = text_list[0].split("\t")
    data = [line.split("\t") for line in text_list[1:]]

    # Create a pandas DataFrame for easier handling
    df = pd.DataFrame(data, columns=header)
    # Split the "Rule" column into "Body" and "Head"
    df[['Body', 'Head']] = df['Rule'].str.split(' => ', expand=True)
    df = df[['Rule', 'Body', 'Head'] + [col for col in df.columns if col not in ['Rule', 'Body', 'Head']]]

    header += ['Body', 'Head']
    # Convert appropriate columns to float and int
    df[header[1]] = df[header[1]].astype(float)  # Head Coverage
    df[header[2]] = df[header[2]].astype(float)  # Std Confidence
    df[header[3]] = df[header[3]].astype(float)  # PCA Confidence
    df[header[4]] = df[header[4]].astype(int)    # Positive Examples
    df[header[5]] = df[header[5]].astype(int)    # Body size
    df[header[6]] = df[header[6]].astype(int)    # PCA Body size

    # Connect to the MySQL server
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )
    cursor = conn.cursor()

    # Create the database if it doesn't exist
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    cursor.execute(f"USE {db_name}")

    # Create the table with the appropriate column names and types
    columns = [
        '`Rule` VARCHAR(1024)',           # Rule (text)
        '`Body` VARCHAR(1024)',            # Body (text)
        "`Head` VARCHAR(255)",              # Head (text)
        '`Head Coverage` FLOAT',         # Head Coverage (float)
        '`Std Confidence` FLOAT',        # Std Confidence (float)
        '`PCA Confidence` FLOAT',        # PCA Confidence (float)
        '`Positive Examples` INT',       # Positive Examples (int)
        '`Body size` INT',               # Body size (int)
        '`PCA Body size` INT',           # PCA Body size (int)
        '`Functional variable` VARCHAR(255)'  # Functional variable (text)
    ]
    # print(f'CREATE TABLE IF NOT EXISTS {table_name} ({", ".join(columns)})')
    cursor.execute(f'DROP TABLE IF EXISTS {table_name}')
    cursor.execute(f'CREATE TABLE {table_name} ({", ".join(columns)})')

    # Insert the data into the table
    for i, row in df.iterrows():
        cursor.execute(
            f'INSERT INTO {table_name} VALUES ({", ".join(["%s"] * len(header))})', 
            tuple(row)
        )
        if i % 1000 == 0:
            print(i)

    # Commit the changes and close the connection
    conn.commit()
    cursor.close()
    conn.close()

    print(f"MySQL database {db_name} with table {table_name} created successfully.")


create_mysql_database(
    text_list=rest, 
    db_name='rules_db', 
    table_name='rules_table_syn', 
    user='root', 
    password='admin123'
)

0
MySQL database rules_db with table rules_table_syn created successfully.


```SELECT Body, Head, `PCA Confidence`, `Head Coverage`
FROM rules_table
ORDER BY `PCA Confidence` DESC, `Head Coverage` DESC```


```
SELECT Body, Head, `PCA Confidence`, `Head Coverage`
FROM rules_table
WHERE Head like "%Relapse%"
ORDER BY (`PCA Confidence` ) DESC
```

# Query rules with PCA higher than a threshold

In [5]:
import mysql.connector
import csv

def query_rules_to_csv(db_name, table_name, user, password, csv_file, PCA_threshold=0.5, host='localhost'):
    try:
        # Connect to the MySQL server
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=db_name
        )
        cursor = conn.cursor()

        # SQL query to fetch rules
        query = f"""SELECT * FROM {table_name} WHERE `PCA Confidence` >= {PCA_threshold} 
        AND `PCA Confidence` < 1.0 
        AND 
        (   (Head not like '%vocab/age%' 
            AND Head not like '%vocab/hasSmokingHabit%' 
            AND Head not like '%vocab/sex%'
            AND Head not like '%vocab/hasStage%'
            AND Head not like '%vocab/locatedInHospital%'
            AND Head not like '%vocab/hasProgressionRelapse%')
            OR (Head LIKE '%vocab/hasProgressionRelapse  Progression%' AND Body LIKE "%vocab/hasStage%" ) 
            OR (Head LIKE '%vocab/hasProgressionRelapse  Relapse%')
        )
        """
        # AND Head not like '%vocab/hasProgressionRelapse  Progression%'  
        """
        If remove the Progression, we have much less patients in KG':  
        1837 -> 1221 
        """ 
        cursor.execute(query)

        # Fetch all rows
        rows = cursor.fetchall()

        # Fetch column names
        column_names = [i[0] for i in cursor.description]

        # Write the result to a CSV file
        with open(csv_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            
            # Write the column headers
            writer.writerow(column_names)
            
            # Write the data rows
            writer.writerows(rows)

        # Close the cursor and connection
        cursor.close()
        conn.close()

        print(f"Data successfully written to {csv_file}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Example usage
query_rules_to_csv(
    db_name='rules_db',
    table_name='rules_table_syn',
    user='root',
    password='admin123',
    csv_file='rules_NSCLP.csv',
    PCA_threshold=0.5 
)


Data successfully written to rules_NSCLP.csv


# Query Rules with PCA Confidence = 1.0 as knowledge for GPT4

In [13]:
import mysql.connector
import csv
import re

family_mapping = {
        "UNK": "UNK", "Father": "M1", "Mother": "F1", "Brother": "M1", "Sister": "F1",
        "Daughter": "F1", "Son": "M1", "Uncle": "M2", "Nephew": "M2", "Grandfather": "M2",
        "Grandmother": "F2", "Aunt": "F2", "Niece": "F2", "Granddaughter": "F2", "Grandson": "M2",
        "Grandgrandfather": "M3", "Grandgrandmother": "F3", "No": "No", "Halfsister": "F2",
        "Halfbrother": "M2", "Female_Cousin": "F3", "Male_Cousin": "M3", "NULL": "NULL", 
        'Granduncle': "M4", 'Greatgrandfather': "M4", 'Greatgrandmother':"F4"}


def transform_line(line):
    # Regex to capture the necessary parts of the line wherever they occur
    pattern = r"(\?a\s+hasFamilyCancerType\s+)(\w+)&(\w+)"
    
    # Function to replace the matched part with the new triple format
    def repl(match):
        subject = "?a"                     # the subject identifier
        family = match.group(2)            # "Brother", "Sister", etc.
        cancer = match.group(3)            # "Prostate", "Lung", etc.

        # Creating the new format: adding a new triple for familyGender
        family_gender = "Male" if "M" in family_mapping[family] else "Female"
        new_triples = f"{subject} familyGender {family_gender} {subject} hasFamilyCancerType {cancer}"
        return new_triples
    
    # Apply the regex substitution
    transformed_line = re.sub(pattern, repl, line)
    return transformed_line


def translate(rule:str):
    translate_dict = {
    "age": "whose age is {}",
    "hasEpisodeType": "whose treatment episode type is {}",
    "hasBio": "who has positive biomarker test result on {}",
    "hasFamilyCancerType": "who has at least one {} that has {} cancer",
    "hasSmokingHabit": "who is a {},",
    "locatedInHospital": "who went to a hospital located in {}",
    "hasProgressionRelapse": "whose cancer has {}",
    "sex": "who is a {}",
    "hasStage": "who is in stage {}"
    }

    rule = rule.replace("http://research.tib.eu/clarify2020/vocab/", "").replace("http://research.tib.eu/clarify2020/entity/", "")
    rule = transform_line(rule)
    return rule
    

def query_rules_to_csv(db_name, table_name, user, password, csv_file, PCA_threshold=0.6, host='localhost'):
    try:
        # Connect to the MySQL server
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=db_name
        )
        cursor = conn.cursor()

        # SQL query to fetch rules
        query = f"""SELECT Rule FROM {table_name} where 
        `PCA Confidence` >= 0.8 
        AND Head NOT LIKE "%age%"
        AND Head NOT LIKE "%gender%"
        """

        # query = f"""SELECT Rule, `PCA Confidence` FROM {table_name} where 
        # `PCA Confidence` >= 0.95 
        # AND Rule NOT LIKE "%vocab/hasDrugSchema%"
        # AND Rule NOT LIKE "%vocab/hasComorbidity%"
        # AND Rule NOT LIKE "%vocab/hasPreviousCancerType%"
        # AND Rule NOT LIKE "%vocab/hasPreviousCancerType%"
        # AND Rule NOT LIKE "%vocab/hasEpisodeType%"
        # AND Rule NOT LIKE "%vocab/hasRace%"
        # AND Rule NOT LIKE "%vocab/hasProgressionRelapse%"
        # AND Rule NOT LIKE "%vocab/locatedInHospital%"
        # AND Rule NOT LIKE "%vocab/hasStage%" """
        # AND Head not like '%vocab/hasProgressionRelapse  Progression%'  
        """
        If remove the Progression, we have much less patients in KG':  
        1837 -> 1221 
        """ 
        cursor.execute(query)

        # Fetch all rows
        rows = cursor.fetchall()
        rows = [(translate(row[0]),) for row in rows]
        
        # Fetch column names
        column_names = [i[0] for i in cursor.description]

        # Write the result to a CSV file
        with open(csv_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            
            # Write the column headers
            writer.writerow(column_names)
            
            # Write the data rows
            writer.writerows(rows)

        # Close the cursor and connection
        cursor.close()
        conn.close()

        print(f"Data successfully written to {csv_file}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Example usage
query_rules_to_csv(
    db_name='rules_db',
    table_name='rules_table_syn',
    user='root',
    password='admin123',
    csv_file='rules_NSCLP_PCA=1.0.csv',
    PCA_threshold=1.0 
)


Data successfully written to rules_NSCLP_PCA=1.0.csv


# Abundant

In [ ]:
import re

def transform_line(line):
    # Regex to capture the necessary parts of the line wherever they occur
    pattern = r"(\?a\s+hasFamilyCancerType\s+)(\w+)&(\w+)"
    
    # Function to replace the matched part with the new triple format
    def repl(match):
        subject = "?a"                     # the subject identifier
        family = match.group(2)            # "Brother", "Sister", etc.
        cancer = match.group(3)            # "Prostate", "Lung", etc.

        # Creating the new format: adding a new triple for familyGender
        new_triples = f"{subject} familyGender {family} {subject} hasFamilyCancerType {cancer}"
        return new_triples
    
    # Apply the regex substitution
    transformed_line = re.sub(pattern, repl, line)
    return transformed_line

# Example usage:
for input_line in """?a  hasFamilyCancerType  Brother&Prostate   => ?a  hasProgressionRelapse  Progression
?a  hasFamilyCancerType  Sister&Lung   => ?a  hasProgressionRelapse  Progression
?a  hasFamilyCancerType  Brother&Head_and_neck   => ?a  hasProgressionRelapse  Progression
?a  hasFamilyCancerType  Brother&Esophagogastric   => ?a  hasProgressionRelapse  Progression
""".split("\n"):
# input_line = "?a  hasFamilyCancerType  Brother&Prostate   => ?a  hasProgressionRelapse  Progression"
    print(transform_line(input_line))

?a familyGender Brother ?a hasFamilyCancerType Prostate   => ?a  hasProgressionRelapse  Progression
?a familyGender Sister ?a hasFamilyCancerType Lung   => ?a  hasProgressionRelapse  Progression
?a familyGender Brother ?a hasFamilyCancerType Head_and_neck   => ?a  hasProgressionRelapse  Progression
?a familyGender Brother ?a hasFamilyCancerType Esophagogastric   => ?a  hasProgressionRelapse  Progression

